In [18]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import json

# Read recipe inputs
disk_usage_25_08 = dataiku.Dataset("disk_usage_25_08")
df = disk_usage_25_08.get_dataframe()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
status    1 non-null object
data      1 non-null object
dtypes: object(2)
memory usage: 96.0+ bytes


In [20]:
df

,status,data
0,success,"{""result"":[{""metric"":{""cluster"":""mutu-100"",""ho..."


In [21]:
for key, value in df["data"].items():
    print(value)
    dict = json.loads(value)
dict

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'result': [{'metric': {'cluster': 'mutu-100',
    'dc': 'dalembert',
    'device': '/dev/sda1',
    'fstype': 'xfs',
    'hostname': 'broker-102',
    'job': 'node',
    'mountpoint': '/'},
   'values': [['1626351300', '15.22182659019522'],
    ['1626352200', '15.29976180210322'],
    ['1626353100', '15.295078631249368'],
    ['1626354000', '15.177431847345417'],
    ['1626354900', '15.2946589580975'],
    ['1626355800', '15.294668496123677'],
    ['1626356700', '15.158045809136752'],
    ['1626357600', '15.238079386804117'],
    ['1626358500', '15.162500067362316'],
    ['1626359400', '15.238103231869559'],
    ['1626360300', '15.238212919170621'],
    ['1626361200', '15.179525444091695'],
    ['1626362100', '15.31636750568073'],
    ['1626363000', '15.316377043706908'],
    ['1626363900', '15.238346451537126'],
    ['1626364800', '15.238222457196798'],
    ['1626365700', '15.179649438432023'],
    ['1626366600', '15.316682260544638'],
    ['1626367500', '15.236624837811831'],
    ['

In [22]:
df = pd.json_normalize(dict["result"])
df = df.explode('values')
df[['Timestamp','CPU_Usage']] = pd.DataFrame(df['values'].tolist(),index=df.index)
df

AttributeError: module 'pandas' has no attribute 'json_normalize'

In [0]:
df.rename(columns={'metric.hostname': 'Broker', 'metric.dc': 'Data_Center','metric.mountpoint':'Disk'}, inplace=True)
df['Broker_Disk'] = df['Broker'] + df['Disk']
df = df.drop(['metric.cluster', 'metric.job','metric.device','metric.fstype','values','Broker','Disk'], axis=1)
df

In [0]:
list(df['Broker_Disk'].unique())

In [0]:
dfx = pd.pivot_table ( df, index=['Timestamp'], columns = df.groupby(['Timestamp']).cumcount().add(1), values = ['CPU_Usage'], aggfunc = 'sum')
dfx.columns = list(df['Broker_Disk'].unique())
dfx = dfx.reset_index()
dfx

In [0]:
df = dfx



# Write recipe outputs
json_2 = dataiku.Dataset("json_2")
json_2.write_with_schema(df)